![alt text](http://greenbookblog.org/wp-content/uploads/2017/02/trump-word-cloud.png)

GROUP DICKY AND CUONG


1. Main quest: Using dataset from kaggle to classification the sentiment of words.
2. Using: TF IDF learn from last week to apply in this problem.

*   First import dataframe for 2 dataset
*   Clean the dataset
*   Creat func clean word
*   Check in the stop word
*   Run logistic regressiob
*   Export file and upload to kaggle

3. Conclusion: TFIDF can recongnize 60% meaning word in stop word within you can clean the data good. 









In [0]:
# import library
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import re
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split


In [0]:
# creat dataframe
df_train = pd.read_csv('/content/train - train.csv')
df_test = pd.read_csv('/content/test - test.csv')

In [0]:
# fill 2 nan row
df_test.fillna('a', inplace = True)

In [0]:
# fill 2 nan row
df_train.fillna('a', inplace = True)

In [0]:
# function clean text
def preprocessor(text):
  text = re.sub("do 'nt", 'do not', text)
  text = re.sub("'d", 'had', text)
  text = re.sub("n't", 'not', text)
  text = re.sub("re", 'are', text)
  text = re.sub("'m", 'am', text)
  text = re.sub("'s", 'is', text)
  text = re.sub("n't", 'not', text)
  text = re.sub("[?,.!@=+-;'`˜`ˆˆ&*]", '', text)
  text = re.sub("ˆ[a-zA-Z]+",'', text.lower())
  return text.strip()

In [0]:
# clean test dataset
test_list = []
for sentence in df_test['Phrase'].values:
  test_list.append(preprocessor(str(sentence)))

In [0]:
# add clean list to dataframe
test_list = pd.Series(test_list)
df_test['Phrase'] = test_list

In [0]:
# clean test dataset
train_test = []
for sentence in df_train['Phrase'].values:
  train_test.append(preprocessor(str(sentence)))

In [0]:
# add clean list to dataframe
train_test = pd.Series(train_test)
df_train['Phrase'] = train_test

In [0]:
# concat 2 columns together
df = pd.concat([df_train['Phrase'], df_test['Phrase']],ignore_index=True)

In [0]:
# count word
count = Counter()
for word in train_test:
  for j in word.split():
    count[j] += 1

In [0]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [0]:
# count word same in stop word
vocab_reduced = Counter()
for i in count.items():
  if i[0] not in stop:
    vocab_reduced[i[0]] += 1

In [0]:
# tokenizer word
porter = PorterStemmer()
def tokenizer_porter(text):
  token = []
  for i in text.split():
    token.append(porter.stem(i))
  
  return token

In [0]:
# create tfidf
tfidf = TfidfVectorizer(stop_words=stop, tokenizer = tokenizer_porter, preprocessor= preprocessor)

In [0]:
# learn values
tfidf.fit(df.values)

In [0]:
#creat 2 series
training = df[:len(df_train)]
testing = df[len(df_train):]

In [0]:
# create training model and vectorizer
training_vector = tfidf.transform(df_train['Phrase'].values)
testing_vector = tfidf.transform(testing.values)

In [0]:
# logistic regression
X = training_vector
y = training['Sentiment']
lg = LogisticRegression()
lg.fit(X, y)

In [0]:
# prediction
y_pred = lg.predict(testing_vector)

In [0]:
# output upto kaggle
def create_output(y_pred):
    data = {
        'PhraseId': df_test['PhraseId'].values,
        'Sentiment': y_pred
    }
    df = pd.DataFrame(data)
    df.to_csv('output.csv', index=False)
    return

create_output(y_pred)